In [ ]:
!pip install evaluate

In [ ]:
# import des librairies
import os
import pandas as pd
import torch
from transformers import AutoTokenizer,AutoModel
from datasets import load_dataset
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch import optim
from sentence_transformers import SentenceTransformer
import numpy as np
from transformers import RobertaModel
from transformers import Trainer
from datasets import Dataset
from transformers import TrainingArguments
import transformers


device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
import transformers
print(transformers.__version__)

4.53.2


In [ ]:
# lire le dataset

import pandas as pd

base_path = "hf://datasets/jingjietan/essays-big5/data/"

splits = {
    'train': f"{base_path}train-00000-of-00001.parquet",
    'validation': f"{base_path}validation-00000-of-00001.parquet",
    'test': f"{base_path}test-00000-of-00001.parquet"
}

df_train = pd.read_parquet(splits['train'])
df_val = pd.read_parquet(splits['validation'])
df_test = pd.read_parquet(splits['test'])

df_train.head()


,O,C,E,A,N,ptype,text,__index_level_0__
0,1,0,0,1,1,19,it is wednesday. I can't wait until friday bec...,774
1,1,1,1,0,1,29,"wow, I want to go talk to the socialist organi...",178
2,1,0,1,1,0,22,"I wish polygamy was still legal. Well, not pol...",1881
3,1,0,1,0,0,20,"Well, lets see . . . I guess the foremost thin...",1563
4,0,1,0,1,1,11,College? I wonder how it will be? I just ...,1594


In [ ]:
df_train = df_train.drop(columns=['C','E','N','A','__index_level_0__','ptype'])
df_val = df_val.drop(columns=['C','E','N','A','__index_level_0__','ptype'])

df_train
df_val

,O,text
0,0,Well I am sitting in my apartment thinkin...
1,0,"Well, I hope I'm doing this assignment correct..."
2,1,"Well, my mother has just gone back to India te..."
3,1,"Gregory benjamin's writhing assingment, s..."
4,1,I'm very worried about many things right ...
...,...,...
390,0,"It is Wednesday, December 8th and a lot has be..."
391,1,"I guess I have a life now, which is great for ..."
392,0,what to write in the paper. hmm pizza is good ...
393,0,"Wow, this is incredibly hard to follow my thou..."


In [ ]:
# garder sur train et val text et O en dataset comme dans la doc
df_train = df_train.rename(columns={"O": "labels"})
df_val = df_val.rename(columns={"O": "labels"})

df_train["labels"] = df_train["labels"].astype(int)
df_val["labels"] = df_val["labels"].astype(int)

dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)
# Tokenize the datasets
dataset_train = dataset_train.map(tokenize, batched=True)
dataset_val   = dataset_val.map(tokenize, batched=True)

Map:   0%|          | 0/1578 [00:00<?, ? examples/s]

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

In [ ]:
dataset_train

Dataset({
    features: ['labels', 'text', 'input_ids', 'attention_mask'],
    num_rows: 1578
})

In [ ]:
# supprimer la colonne text
dataset_train = dataset_train.remove_columns("text")
dataset_val = dataset_val.remove_columns("text")

In [ ]:
dataset_train

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1578
})

In [ ]:
from transformers import AutoModelForSequenceClassification, RobertaConfig

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,   # Assuming binary classification, adjust if needed
    problem_type="single_label_classification",
    return_dict=True,
    hidden_dropout_prob=0.2,
    #attention_probs_dropout_prob=0.0 ,
    #classifier_dropout=0.0
)

model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    config=config
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
import evaluate


accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1.compute(predictions=predictions, references=labels)["f1"],
    }

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta_ocean_classifier",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.1,
    logging_dir="./logs",
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=1,
    report_to="none"
)

In [ ]:
print(dataset_train[0])

{'labels': 1, 'input_ids': [0, 405, 16, 18862, 46836, 4, 38, 64, 75, 2067, 454, 6664, 21746, 142, 38, 524, 164, 184, 7, 192, 1518, 261, 4, 38, 2649, 123, 98, 203, 4, 38, 64, 75, 2067, 7, 192, 123, 4, 80, 55, 360, 4, 42, 34, 57, 10, 182, 251, 80, 688, 4, 86, 3974, 182, 5764, 259, 4, 38, 33, 10, 319, 9, 481, 86, 15, 127, 1420, 77, 38, 524, 45, 11, 1380, 4, 1380, 4, 16797, 1380, 4, 16797, 16, 1531, 98, 444, 4, 24, 269, 3168, 162, 6, 8, 8546, 4, 7670, 858, 428, 4218, 16, 6269, 4, 7285, 80, 2345, 9, 8265, 162, 600, 4, 141, 524, 38, 164, 7, 2145, 70, 9, 167, 1110, 4, 38, 399, 75, 190, 2073, 2600, 24, 142, 38, 399, 75, 1346, 24, 4, 53, 38, 197, 33, 44949, 281, 1780, 28648, 26, 14, 24, 21, 2679, 4, 37, 21, 2758, 162, 59, 141, 51, 847, 103, 233, 9, 10, 4758, 18, 2900, 66, 11, 41, 9280, 4, 14, 16, 7735, 4, 5, 2129, 4758, 4, 28648, 16, 7735, 350, 4, 38, 460, 5170, 114, 37, 3829, 162, 4, 37, 64, 28, 98, 1266, 77, 97, 82, 32, 198, 53, 98, 2579, 77, 24, 16, 95, 5, 80, 9, 201, 4, 38, 222, 619, 1256, 

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.693400,0.689524,0.564557,0.695035
2,0.680600,0.672434,0.582278,0.469453
3,0.662600,0.665096,0.605063,0.493506
4,0.632800,0.660789,0.597468,0.530973
5,0.611000,0.666231,0.594937,0.526627


TrainOutput(global_step=495, training_loss=0.6560586332070707, metrics={'train_runtime': 510.2741, 'train_samples_per_second': 15.462, 'train_steps_per_second': 0.97, 'total_flos': 1037973113395200.0, 'train_loss': 0.6560586332070707, 'epoch': 5.0})